<a href="https://colab.research.google.com/github/Svetorus/Neuronets/blob/master/Neuronet_HW_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Char-based text generation with LSTM

In [0]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [0]:
from google.colab import files

uploaded = files.upload()

Saving Память_света.txt to Память_света (1).txt


In [0]:
TRAIN_TEXT_FILE_PATH = 'Память_света.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', 'о', 'а', 'е', 'н', 'и', 'т', 'л', 'с', 'р', 'в', 'д', 'м', 'у', 'к', 'п', ',', 'ь', 'я', 'ы', '.', 'з', 'б', 'г', 'ч', 'й', '—', 'ж', 'х', 'ш', '\n', 'ё', 'ю', 'э', 'щ', 'О', 'П', 'Т', 'ц', 'В', 'А', 'С', 'Н', 'М', '?', 'И', 'Р', 'Э', 'Д', 'К', '-', 'Б', 'Л', 'Я', 'Е', '…', 'ф', '!', 'Г', 'Ч', 'У', '«', '»', 'З', '’', ':', 'Х', 'ъ', 'Ш', 'Ж', 'Ф', ';', 'ó', 'Ю', 'Ц', '0', 'Ь', '2', '7', '\u2003', '́']


In [0]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [0]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [0]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=2):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=4)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 10000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
        print(predicted_text)

Loss: 3.363550109863281
    оеи р   о     оо  н    о  а         н  он  роел н     е а     ое   ра    а о    ол   лл к  р       ии  е а         о е          л  у  м     о     н    и  о о   о ио  о   ,о л  о о   ю      наоо    
Loss: 3.3015382957458494
   ои        оас а     а     о       л   ор      ат       оо                    е   ое        т         а         н          а  а   о            а  т     а      н   ос  е      е о   о     а     са     
Loss: 3.303658137321472
  он   н от                    н    е ное            ноло т  о  н          е     о           в        о      сна  н          а      а    а    о       о а      таоо ан оое   е о  ан  я        е тр  н  в
Loss: 3.304134936332703
             о е и        н       о          р       не   ар е      е    тое          он       а  а   на н а и     о             и ооо      о   о    и              рн  о   ра и    а    о             а 
Loss: 3.3009567403793336
  оа оаот   н   с   е    а  е        о      у       оо о   н  нноо а  

In [0]:
model.eval()

print(evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=1000, 
    start_text='. '
    )
)

. подобное стать на остальные не всего последнее время не поставила себя на положил в нём с тем, что и всё в сторону с своим под сторону на него не последнее собрались в сторону присоспешно на него просто с политики на политики на то, что собирался в передала с собой собой положил в заплатила она в сторону не стал на столом с от него подумал Андрол. — В прошлом. — Во своей всё в собой последние от него ударов в под руками с открыла в поверхности подобной все всего случае в положил в этом и стороне собрали себя в него поняла он подобной на него не смогут столько голову показалось подобное от возможно, он присомачал Перрин, которые помощью и присомадовиться с себя на него поднял в том, что собираюсь в сторону в какой из собой в сторону просто проверить подобной на то спросил Андрол. — Старалась на него поставить продолжила Эгвейн. — Он посмотрела под том, что они могли не стала столько поднялся и она подобного не стал так пробралась с собой прислушался от него просто не подобное поставит

тебе они не завершивали в комнате. — Просто под тебе подобного возвращением все не пришлось в поле собралась на него вернулся к нему подобное обратно подальше с ним подобного пополнила на себя на то, что собирается с неё стоял в несколько собственным от другой своим должно было потому было обратно в сторону прошло присомадовил в политики принялись в одной приманки приказать в сторону на его собирается слишком подумал Мэт и волось в конце собирался в себя на него вызвали в последнее обратно не было собрались подобное не стала содрогнулся в него последние сторону положил в сторону, что всё воздух в себя на то, что она не поставить на подобным поле может подальше последнее время с продолжил он.
 — Может, в под том, что они собирался повернулся в повернулся в собственных своим противника было не стала на собой в конце собралась на прошло вы воздуха с другом в конце на сторону с подобное направлять с другом, — под ним стал меня подобные на последнее столом положил положила своим подобного н

тебе не продолжила она подобное волос на сторону с их до него не ответил Андрол. — На собственные подобного подобных поставить троллоков в под подобного ответила она, — поднял на сторону повернулся и приказать может следует стояла подобное сторону в него с тем потоком. Он отправился в сторону в том просто собираюсь пристально подобной на самом деле подобного не продолжила она под лезвался Талманес последние сторону под своих на присоспешно не стал подобных сторону подобные положила своим под него, который сказала Певара не последние воспользовало и собирался от подобное ворота в собой троллоков в собственных присостально королева под контроль подальше не просто волосомнерая от столько всего, произнёс Эмарин.
 — Ты под сторону странно в том, что он стреливать в порядке, — продолжил Бэйрд под вот в которых другой последние подобной собирается подобной в своей подобное продолжила она с ним всё настолько всегда подальше проводила должно было собирается с ним последнее стал собственные и ст



ним на него не присомнила стояли подумал Андрол и на сторону подобное ворота в сторону на то, что бы не против него всего подобного собрался в позволила становился в политики за ним. — Но подобного обратились на него подняли в положила она все просто не поставил друг с подобного с меня на положил в сторону с подобного всё в положил свои возвращения на должны положил столько положил свои пристально прислушался с своим от него не против немного всего столько собирается убить на то своим под странно подвергали в состоянии с ним на больше с под ней в него как поднялся с крайней мере, продолжила Илэйн. — Хорошо, — ответил Ранд. — Он подальше постарался за собирался с положил он с несколько против всегда в сторону с собой и он подобные столько не стала странно подобные присоспешно подобной на него развернула от своим волна просто не подходящий на собственных просто не просто под темноволосой в одной собирается на подобного не подобное подумала она, — произнёс Ранд. — Я способных на то, что 